In [1]:
import pandas as pd
import AdsAbsRequester
from collections import Counter

In [2]:
delCols=['page_count', 'author', 'aff', 'citation_count', 'author_count','bibstem', 'title']
def responseToPandas(response: list):
    df=pd.DataFrame(columns=['pubdate', 'keyword'])
    for rec in response:
        r=rec
        for c in delCols:
            r.pop(c,None)
        df=df.append(r,ignore_index=True)
    return df

In [3]:
token="KDaPKeB4q7noKbg00u3gjlqszxYEWQ94YguJ0OmR"
requester=AdsAbsRequester.Requester(token,'testRequest')

In [4]:
years=range(2010,2021)

In [6]:
def request(year: int, additional: str)->int:
    searchCriteria=('year:'+str(year),'database:astronomy',
                    'property:refereed','abs:"machine learning"', additional)
    requester.setParams(searchCriteria)
    requester.loadData()
    count=len(requester.response.json()['response']['docs'])
    return count

In [13]:

additionals=['Milky Way','exoplanet','gravitational wave',
             'photometric redshift','galaxy classification','star/galaxy',
            'lensing','transient','Sun']
statByYears=pd.DataFrame(columns=['year','ML']+additionals)
for year in years:
    print(year)
    counts={'year':year}
    c=request(year,'')
    counts['ML']=c
    for add in additionals:
        c=request(year,'abs:"'+add+'"')
        counts[add]=c
    statByYears=statByYears.append(counts,ignore_index=True)

2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020


In [14]:
statByYears['total']=statByYears.iloc[:, 2:].sum(axis=1)
statByYears

,year,ML,Milky Way,exoplanet,gravitational wave,photometric redshift,galaxy classification,star/galaxy,lensing,transient,Sun,total
0,2010,14,1,0,0,1,0,0,0,1,4,7.0
1,2011,13,3,0,0,2,1,0,2,0,2,10.0
2,2012,30,3,1,0,3,1,1,2,3,4,18.0
3,2013,27,2,0,3,3,0,0,0,5,6,19.0
4,2014,40,6,0,1,4,1,1,2,4,7,26.0
5,2015,69,9,2,4,7,1,5,4,9,16,57.0
6,2016,82,9,5,2,17,3,4,5,8,15,68.0
7,2017,94,10,2,6,12,4,3,10,9,17,73.0
8,2018,173,23,9,7,13,5,5,12,13,41,128.0
9,2019,280,22,14,14,15,5,7,25,26,63,191.0


In [16]:
statByYears.to_csv('topicsFromAbstract.csv',index=False)